In [1]:
import sys

sys.path.append('..')

from spinesUtils import *
import pandas as pd

In [2]:
df = dataloader('/Users/guobingming/Downloads/dataset.csv', turbo_method='dask', transform2low_mem=False)

In [3]:
df = df.iloc[:1000000, :]
df = df[select_numeric_cols(df)]
df.columns

Index(['vip_end_days', 'is_bought_today', 'unwatching_days', 'watch_last_nums',
       'daydiff', 'before_last_date', 'vod_play_pv', 'vod_cnt', 'vod_vv',
       'vod_over_pv',
       ...
       'next_login_monthdiff', 'next_login_weekdiff', 'next_daydiff',
       'next_login_vip_valid_month', 'lost_7_remain_ds', 'lost_15_remain_ds',
       'lost_7_remain_days_td', 'lost_15_remain_days_td',
       'next_cumulative_login_ds', 'next_login_7ds_freq'],
      dtype='object', length=193)

In [4]:
y_col = 'is_losing_user'

In [5]:
x_cols = get_x_cols(df, y_col, exclude_columns=['dt', 'userid'])

In [6]:
classify_samples_dist(df, y_col)

,sample_count,rate
is_losing_user,,
0.0,953048,95.3%
1.0,46952,4.7%


In [7]:
from lightgbm import LGBMClassifier
classifier = BinaryBalanceClassifier(meta_estimators=[LGBMClassifier(), LGBMClassifier()])

In [8]:
classifier.fit(df[x_cols], df[y_col], threshold_search_set=(df[x_cols], df[y_col]))

18:28:12 2023-09-19 - [Main step] X type is pandas dataframe, y type is pandas series.
18:28:12 2023-09-19 - [Category columns] 
18:28:12 2023-09-19 - [Main step] Prepare to fit.
18:28:12 2023-09-19 - [Model L1] Training samples shape is (1000000, 192), sample size ratio is {0.0: 0.95, 1.0: 0.05}
18:28:12 2023-09-19 - [Main step] Model L1 fitted.
18:28:13 2023-09-19 - [Main step] Datasets split for model L2.
18:28:13 2023-09-19 - [Model L2] Training samples shape is (93904, 192), sample size ratio is {0.0: 0.5, 1.0: 0.5}
18:28:15 2023-09-19 - [Main step] Model L2 fitted.
18:28:16 2023-09-19 - [Main step] Datasets split for model L3.
18:28:16 2023-09-19 - [Model L3] Training samples shape is (273731, 192), sample size ratio is {0.0: 0.86, 1.0: 0.14}
18:28:21 2023-09-19 - [Main step] Model L3 fitted.
18:28:21 2023-09-19 - [Main step] start to search probability threshold...
18:28:21 2023-09-19 - Automatically searching...
18:28:23 2023-09-19 - [current loop] try threshold: 0.0, max f1 sc

In [9]:
classifier.auto_threshold

frozendict.frozendict({'Model L2': 0.8057999999999998, 'Model L3': 0.17319999999999988})

In [10]:
from sklearn.metrics import f1_score, recall_score, precision_score
(
    'f1:', f1_score(df[y_col], classifier.predict(df[x_cols])), 
    'recall:', recall_score(df[y_col], classifier.predict(df[x_cols])), 
    'precision:', precision_score(df[y_col], classifier.predict(df[x_cols]))
)

18:29:24 2023-09-19 - [Main step] Prepare to predict. Predict mode:soft
18:29:24 2023-09-19 - [Main step] Model L1 has been predicted.
18:29:26 2023-09-19 - [Main step] Model L2 has been predicted.
18:29:27 2023-09-19 - [Main step] Model L3 has been predicted.
18:29:28 2023-09-19 - [Main step] Prepare to predict. Predict mode:soft
18:29:28 2023-09-19 - [Main step] Model L1 has been predicted.
18:29:29 2023-09-19 - [Main step] Model L2 has been predicted.
18:29:30 2023-09-19 - [Main step] Model L3 has been predicted.
18:29:31 2023-09-19 - [Main step] Prepare to predict. Predict mode:soft
18:29:31 2023-09-19 - [Main step] Model L1 has been predicted.
18:29:32 2023-09-19 - [Main step] Model L2 has been predicted.
18:29:33 2023-09-19 - [Main step] Model L3 has been predicted.


('f1:',
 0.34715722444028907,
 'recall:',
 0.36575651729425795,
 'precision:',
 0.330358001654387)

In [11]:
from sklearn.metrics import f1_score, recall_score, precision_score
lgb = LGBMClassifier()
lgb.fit(df[x_cols], df[y_col])
(
    'f1:', f1_score(df[y_col], lgb.predict(df[x_cols])), 
    'recall:', recall_score(df[y_col], lgb.predict(df[x_cols])), 
    'precision:', precision_score(df[y_col], lgb.predict(df[x_cols]))
)

('f1:',
 0.08120122493332016,
 'recall:',
 0.04376810359516101,
 'precision:',
 0.561015561015561)